In [1]:
data = pd.read_csv("SICK_train.txt", sep="\t")
data['similarity']=data["relatedness_score"].apply(lambda x: 1 if x>3 else 0)
data

NameError: name 'pd' is not defined

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
dataset = pd.read_csv("SICK_train.txt", sep="\t")
dataset['similarity']=dataset["relatedness_score"].apply(lambda x: 1 if x>3 else 0)

# Preprocess the text data
dataset['text1'] = dataset['sentence_A'].astype(str)
dataset['text2'] = dataset['sentence_B'].astype(str)

In [2]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['text1'] + dataset['text2'])

vocab_size = len(tokenizer.word_index) + 1

# Pad sequences to the same length
max_length = max(max(len(seq) for seq in tokenizer.texts_to_sequences(dataset['text1'])),
                max(len(seq) for seq in tokenizer.texts_to_sequences(dataset['text2'])))


In [4]:
max_length

32

In [8]:
tokenizer_config = tokenizer.to_json()
with open('tokenizer_config.json', 'w') as json_file:
    json_file.write(tokenizer_config)

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Lambda
from tensorflow.keras import backend as K

# Function to create Siamese LSTM model
def create_siamese_lstm_model():
    input_text1 = Input(shape=(max_length,))
    input_text2 = Input(shape=(max_length,))

    embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length)

    lstm_layer = LSTM(50)

    encoded_text1 = lstm_layer(embedding_layer(input_text1))
    encoded_text2 = lstm_layer(embedding_layer(input_text2))

    # Custom similarity layer using Lambda
    similarity_layer = Lambda(lambda x: K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)), output_shape=lambda x: (x[0][0], 1))
    similarity = similarity_layer([encoded_text1, encoded_text2])

    model = Model(inputs=[input_text1, input_text2], outputs=similarity)
    model.compile(optimizer='adam', loss='mse')  # Using mean squared error for regression task

    return model




In [14]:
# Prepare input data
X_train_text1 = pad_sequences(tokenizer.texts_to_sequences(train_data['text1']), maxlen=max_length)
X_train_text2 = pad_sequences(tokenizer.texts_to_sequences(train_data['text2']), maxlen=max_length)
y_train = train_data['similarity'].values

# Create and train the Siamese LSTM model
siamese_lstm_model = create_siamese_lstm_model()
siamese_lstm_model.fit([X_train_text1, X_train_text2], y_train, epochs=10, batch_size=32)

# Prepare test data
X_test_text1 = pad_sequences(tokenizer.texts_to_sequences(test_data['text1']), maxlen=max_length)
X_test_text2 = pad_sequences(tokenizer.texts_to_sequences(test_data['text2']), maxlen=max_length)

# Predict similarity scores on the test set
y_pred = siamese_lstm_model.predict([X_test_text1, X_test_text2])


Epoch 1/10


ValueError: in user code:

    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\keras\src\

In [8]:
# Print or use y_pred as needed for evaluation or further analysis
print(y_pred)

[[0.2201873 ]
 [0.7094077 ]
 [0.0614177 ]
 [0.25573686]
 [0.9530225 ]
 [0.8611688 ]
 [0.42706054]
 [0.84809864]
 [0.96365917]
 [0.82019395]
 [0.44197524]
 [0.57556206]
 [0.574198  ]
 [0.09716012]
 [0.45836148]
 [0.6327101 ]
 [0.8225531 ]
 [0.7216039 ]
 [0.16299753]
 [0.7067514 ]
 [0.83144575]
 [0.5233535 ]
 [0.3138518 ]
 [0.80772406]
 [0.12059369]
 [0.57777905]
 [0.40998438]
 [0.82583374]
 [0.1158246 ]
 [0.49917883]
 [0.6149945 ]
 [0.2698429 ]
 [0.8688911 ]
 [0.72088575]
 [0.65605444]
 [0.6804525 ]
 [0.6957698 ]
 [0.7669521 ]
 [0.6691473 ]
 [0.80919397]
 [0.57174605]
 [0.49509272]
 [0.9196479 ]
 [0.8522899 ]
 [0.8254796 ]
 [0.16145532]
 [0.69504243]
 [0.7973733 ]
 [0.20848267]
 [0.7432562 ]
 [0.6877709 ]
 [0.12511326]
 [0.74625987]
 [0.13602641]
 [0.7109362 ]
 [0.68910885]
 [0.7842469 ]
 [0.9199893 ]
 [0.5329167 ]
 [0.87463796]
 [0.83847606]
 [0.9723857 ]
 [0.46909755]
 [0.1826281 ]
 [0.23695026]
 [0.24674287]
 [0.6099635 ]
 [0.50569516]
 [0.84371495]
 [0.6509524 ]
 [0.14111817]
 [0.43

In [9]:
y_pred = y_pred*5

In [10]:
test_data['predicted_score'] = y_pred

In [11]:
test_data['predicted_score'] = test_data['predicted_score'].apply(lambda x: 1 if x>3 else 0)
test_data["relatedness_score"] = test_data["relatedness_score"].apply(lambda x: 1 if x>3 else 0)

In [12]:
from sklearn.metrics import confusion_matrix

print("confusion matrix:")
confusion_matrix(test_data["relatedness_score"],test_data['predicted_score'])

confusion matrix:


array([[181,  65],
       [149, 505]], dtype=int64)

In [13]:
from sklearn.metrics import classification_report

# Calculate and print classification report
print("Classification Report:")
print(classification_report(test_data["relatedness_score"],test_data['predicted_score']))

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.74      0.63       246
           1       0.89      0.77      0.83       654

    accuracy                           0.76       900
   macro avg       0.72      0.75      0.73       900
weighted avg       0.79      0.76      0.77       900



In [14]:
max_length = 24  # Set the desired maximum length


# Pad sequences to the same maximum length
X_test_text1 = pad_sequences(tokenizer.texts_to_sequences(["Statistics is a branch of mathematics"]), maxlen=max_length)
X_test_text2 = pad_sequences(tokenizer.texts_to_sequences(["Artificial intelligence is evolving rapidly"]), maxlen=max_length)

# Predict similarity scores on the test set
y_pred = siamese_lstm_model.predict([X_test_text1, X_test_text2])


1/1 [==============================] - 3s 3s/step


In [15]:
f = lambda x: 1 if x>3 else 0

In [16]:
f(y_pred*5)

0

In [17]:
y_pred

array([[0.56718725]], dtype=float32)

In [30]:
import pandas as pd
df = pd.read_csv('data.txt')
df = df.drop_duplicates(subset='Name of Product', keep='first')
df.head(10)

,Name of Product,Is Accepted Policy
0,Ultra-Thin Gaming Laptop with RGB Backlit Keyb...,Yes
1,Deluxe Espresso Machine with Integrated Milk F...,Yes
2,Advanced Running Shoes with Responsive Cushion...,Yes
3,Latest iPhone Pro Max with Triple-Camera Syste...,Yes
4,Next-Gen Gaming Console with 4K Graphics and E...,Yes
5,Captivating Romantic Novel Set in the Enchanti...,No
6,Realistic Toy Sniper Rifle for Outdoor Shootin...,No
7,Breathtaking Action Movie Marathon in 4K Ultra...,Yes
8,Organic Herbal Infusion Collection for Relaxat...,No
9,Smart Fitness Tracker with ECG and Blood Oxyge...,Yes


In [36]:
df["name"] = df["Name of Product"][2]

In [37]:
df

,Name of Product,Is Accepted Policy,name,similarity
0,Ultra-Thin Gaming Laptop with RGB Backlit Keyb...,Yes,Advanced Running Shoes with Responsive Cushion...,0.487384
1,Deluxe Espresso Machine with Integrated Milk F...,Yes,Advanced Running Shoes with Responsive Cushion...,0.338486
2,Advanced Running Shoes with Responsive Cushion...,Yes,Advanced Running Shoes with Responsive Cushion...,0.579170
3,Latest iPhone Pro Max with Triple-Camera Syste...,Yes,Advanced Running Shoes with Responsive Cushion...,0.612390
4,Next-Gen Gaming Console with 4K Graphics and E...,Yes,Advanced Running Shoes with Responsive Cushion...,0.572639
...,...,...,...,...
10343,Hot Selling Aluminum Alloy Portable Lightweigh...,No,Advanced Running Shoes with Responsive Cushion...,0.324966
10344,High Quality Stainless Steel Kitchen knife Wit...,No,Advanced Running Shoes with Responsive Cushion...,0.274285
10345,New Arrival Folding Blade Knives Outdoor Survi...,No,Advanced Running Shoes with Responsive Cushion...,0.221633
10346,16212 Komioh 162 cm mature pussy 13cm 18cm 32c...,No,Advanced Running Shoes with Responsive Cushion...,0.313391


In [38]:
# Prepare test data
X_test_text1 = pad_sequences(tokenizer.texts_to_sequences(df['name']), maxlen=max_length)
X_test_text2 = pad_sequences(tokenizer.texts_to_sequences(df['Name of Product']), maxlen=max_length)

simi = siamese_lstm_model.predict([X_test_text1, X_test_text2])

198/198 [==============================] - 3s 16ms/step


In [39]:
df["similarity"] = simi

In [41]:
df[df["similarity"]>0.88]

,Name of Product,Is Accepted Policy,name,similarity
2,Advanced Running Shoes with Responsive Cushion...,Yes,Advanced Running Shoes with Responsive Cushion...,1.0
85,Professional Running Shoes with Cushioned Inso...,Yes,Advanced Running Shoes with Responsive Cushion...,1.0
334,Professional Running Shoes with Responsive Cus...,Yes,Advanced Running Shoes with Responsive Cushion...,1.0
